In [1]:
import numpy as np
import matplotlib.pyplot as plt
import copy
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
%matplotlib inline

In [37]:
# built linear regression model
class LinearRegression(object):
    def __init__(self):
        self.input = None
        self.output = None
        self.n_features = None
        self.n_targets = None
        
        self.coef_ = None
    
    def cos_function(self, predictions, lables):
        return 0.5 * np.square(lables - predictions).mean()
      
    # training linear regression model
    def fit(self, X, Y, max_iters=1000000, lr=0.01, verbose=False, print_freq=20, tol=1e-6, lambd=1e-2):
        self.input = copy.deepcopy(X)
        self.output = copy.deepcopy(Y)
        n_samples = X.shape[0]
        
        X = np.matrix(X)
        Y = np.matrix(Y.reshape(n_samples, -1))
        
        self.n_features = X.shape[1]
        self.n_targets = Y.shape[1]
        
        self.coef_ = np.mat(np.zeros([self.n_features,1]))
        self.intercept_ = 0.
        
        lr0 = copy.deepcopy(lr)
        decay_rate = 0.9
        decay_period = 2
        
        pre_loss = float('inf')
        for iter in range(1, max_iters+1):
            # learning rate decay 
            if iter > decay_period:
                lr = lr0 / (1 + decay_rate * (iter - decay_period))
            Y_pred = self.predict(X)

            self.coef_ = self.coef_ - lr * np.dot(X.T, (Y_pred - Y)) / n_samples
            self.intercept_ = self.intercept_ - lr * (Y_pred-Y).mean()
            loss = self.cos_function(Y_pred, Y)

            if verbose and (iter % print_freq == 0 or iter == max_iters):
                print('iteration:%d\t loss:%lf' %(iter, loss))
                if pre_loss - loss < tol:
                    break
                pre_loss = loss
            
        
    def predict(self, X):
        return np.dot(X, self.coef_) + self.intercept_
    
    # 'R2 score'
    def R2_score(self, X, Y):
        X = np.matrix(X)
        Y = np.matrix(Y.reshape(-1, self.n_targets))
        Y_pred = self.predict(X)
        u = np.sum(np.square(Y - Y_pred))
        v = np.sum(np.square(Y - np.mean(Y)))
        return 1 - (u/v)
    
    # 'MSE'
    def MSE(self, X, Y):
        X = np.matrix(X)
        Y = np.matrix(Y.reshape(-1, self.n_targets))
        Y_pred = self.predict(X)
        return np.square(Y - Y_pred).mean()
    

In [4]:
# create regression date to test linear regression model
from sklearn import datasets
x, y = datasets.make_regression(n_samples=1000, n_features=3, n_targets=4, noise=13)

In [5]:
LR = LinearRegression()
LR.fit(x, y, lr=1, max_iters=500)
LR.score(x, y)

0.981428547596185

In [38]:
# load FinTech data
path_data = './data/Fintech_daily_data.csv'
df = pd.read_csv(path_data)

In [39]:
df.describe()

,DXYUSD_Open,DXYUSD_High,DXYUSD_Low,DXYUSD_Close,DXYUSD_AdjClose,DXYUSD_Volume,World_Index_Open,World_Index_High,World_Index_Low,World_Index_Close,...,TNXTbill_Open,TNXTbill_High,TNXTbill_Low,TNXTbill_Close,TNXTbill_AdjClose,TNXTbill_Volume,BTC_Price,BTC_market_cap,BTC_volume,Sentiment_24h
count,1917.000000,1917.000000,1917.000000,1917.000000,1917.000000,1.917000e+03,1933.000000,1933.000000,1933.000000,1933.000000,...,1917.000000,1917.000000,1917.000000,1917.000000,1917.000000,1917.0,2803.000000,2.802000e+03,2.803000e+03,2805.000000
mean,93.021101,93.325806,92.748701,93.043193,93.043193,1.214919e+04,80.024273,80.360797,79.595173,79.982856,...,2.145511,2.169513,2.121338,2.145574,2.145574,0.0,4125.471133,7.230983e+10,7.606231e+09,0.418845
std,6.427827,6.443005,6.370153,6.410710,6.410710,3.856245e+05,11.289516,11.293338,11.294457,11.299139,...,0.639394,0.637413,0.639804,0.637501,0.637501,0.0,4721.972529,8.543778e+10,1.270574e+10,0.396992
min,79.139999,79.239998,78.910004,79.139999,79.139999,0.000000e+00,59.000000,59.000000,58.980000,59.000000,...,0.484000,0.538000,0.398000,0.499000,0.499000,0.0,67.809000,7.713681e+08,0.000000e+00,-1.083872
25%,90.250000,90.570000,90.010002,90.320000,90.320000,0.000000e+00,71.129997,71.379997,70.709999,71.040001,...,1.817000,1.842000,1.786000,1.815000,1.815000,0.0,416.802000,5.835826e+09,6.412286e+07,0.148308
50%,95.169998,95.500000,94.910004,95.150002,95.150002,0.000000e+00,76.500000,76.970001,76.000000,76.620003,...,2.271000,2.293000,2.243000,2.268000,2.268000,0.0,1082.016823,1.710097e+10,1.275837e+09,0.214014
75%,97.410004,97.680000,97.160004,97.419998,97.419998,0.000000e+00,89.190002,89.489998,88.739998,89.190002,...,2.610000,2.631000,2.585000,2.605000,2.605000,0.0,7534.740259,1.330727e+11,6.834460e+09,0.620860
max,103.260002,103.820000,103.029999,103.290001,103.290001,1.429000e+07,112.940002,112.940002,112.290001,112.410004,...,3.239000,3.248000,3.233000,3.234000,3.234000,0.0,29022.418395,5.394380e+11,8.140669e+10,4.667784


In [40]:
# remove rows with empty values
df.dropna(inplace=True)
df.describe()

,DXYUSD_Open,DXYUSD_High,DXYUSD_Low,DXYUSD_Close,DXYUSD_AdjClose,DXYUSD_Volume,World_Index_Open,World_Index_High,World_Index_Low,World_Index_Close,...,TNXTbill_Open,TNXTbill_High,TNXTbill_Low,TNXTbill_Close,TNXTbill_AdjClose,TNXTbill_Volume,BTC_Price,BTC_market_cap,BTC_volume,Sentiment_24h
count,1914.000000,1914.000000,1914.000000,1914.000000,1914.000000,1.914000e+03,1914.000000,1914.000000,1914.000000,1914.000000,...,1914.000000,1914.000000,1914.000000,1914.000000,1914.000000,1914.0,1914.000000,1.914000e+03,1.914000e+03,1914.000000
mean,93.027957,93.332435,92.755319,93.049765,93.049765,1.216823e+04,79.997738,80.335481,79.567205,79.956865,...,2.145385,2.169404,2.121249,2.145509,2.145509,0.0,4099.635902,7.183183e+10,7.692876e+09,0.421149
std,6.427832,6.443079,6.370284,6.410779,6.410779,3.859264e+05,11.252597,11.256787,11.257036,11.262245,...,0.639717,0.637739,0.640107,0.637803,0.637803,0.0,4673.566240,8.452504e+10,1.285617e+10,0.395252
min,79.139999,79.239998,78.910004,79.139999,79.139999,0.000000e+00,59.000000,59.000000,58.980000,59.000000,...,0.484000,0.538000,0.398000,0.499000,0.499000,0.0,76.391500,8.700820e+08,0.000000e+00,-0.126262
25%,90.269997,90.589996,90.062498,90.324999,90.324999,0.000000e+00,71.122501,71.379997,70.702497,71.045000,...,1.816250,1.842250,1.786500,1.815000,1.815000,0.0,416.865000,5.806414e+09,7.005387e+07,0.145776
50%,95.174999,95.510002,94.915001,95.155003,95.155003,0.000000e+00,76.490002,76.940002,75.990002,76.610000,...,2.271000,2.293500,2.244000,2.268000,2.268000,0.0,1076.184290,1.706444e+10,1.257546e+09,0.210078
75%,97.417500,97.680000,97.160004,97.419998,97.419998,0.000000e+00,89.187502,89.487499,88.699997,89.157504,...,2.610000,2.631000,2.584250,2.605000,2.605000,0.0,7596.436701,1.330727e+11,7.143630e+09,0.618245
max,103.260002,103.820000,103.029999,103.290001,103.290001,1.429000e+07,112.940002,112.940002,112.290001,112.410004,...,3.239000,3.248000,3.233000,3.234000,3.234000,0.0,28837.288529,5.359673e+11,8.140669e+10,4.652801


In [41]:
df_input = df.drop(['BTC_Price', 'BTC_market_cap', 'BTC_volume', 'Date'], axis=1)
df_ouput = df[['BTC_Price']]
df_input.shape, df_ouput.shape

((1914, 61), (1914, 1))

In [42]:
# 75% data is the train set, and 25% test set
X_train, X_test, Y_train, Y_test = train_test_split(df_input.values, df_ouput.values, test_size=0.25, random_state=1)
X_train.shape, Y_train.shape, X_test.shape, Y_test.shape

((1435, 61), (1435, 1), (479, 61), (479, 1))

In [26]:
X_train, Y_train = df_input.values, df_ouput.values

In [27]:
X_test = copy.deepcopy(X_train)
Y_test = copy.deepcopy(Y_train)

In [43]:
# Normalized data
X_scaler = StandardScaler()
Y_scaler = StandardScaler()
X_scaler = X_scaler.fit(X_train)
X_train = X_scaler.transform(X_train)
X_test = X_scaler.transform(X_test)

Y_scaler = Y_scaler.fit(Y_train)
Y_train = Y_scaler.transform(Y_train)
Y_test = Y_scaler.transform(Y_test)

In [44]:
X_train.shape, Y_train.shape, X_test.shape, Y_test.shape

((1435, 61), (1435, 1), (479, 61), (479, 1))

In [30]:
X_train.mean(), X_train.std()

(7.692464652181225e-17, 0.9834699358669274)

In [50]:
LR = LinearRegression()
LR.fit(X_train, Y_train, verbose=False, max_iters=10000, lr=1, print_freq=50)
LR.R2_score(X_test, Y_test)

0.8556971103570404

In [52]:
LR.MSE(X_train, Y_train)

0.14443553741162193

In [51]:
LR.MSE(X_test, Y_test)

0.15726686289074457

In [34]:
df_coef = pd.DataFrame(columns=['variable', 'coefficient'])
df_coef['variable'] = df_input.columns
df_coef['coefficient'] = LR.coef_
df_coef['abs_coefficient'] = df_coef['coefficient'].abs()
# sort by absolute value of coefficient
pd.set_option('display.max_rows',None)
df_coef.sort_values(by=['abs_coefficient'], ascending=False)

,variable,coefficient,abs_coefficient
8,World_Index_Low,0.123202,0.123202
7,World_Index_High,0.121979,0.121979
9,World_Index_Close,0.121439,0.121439
6,World_Index_Open,0.121385,0.121385
60,Sentiment_24h,0.113278,0.113278
10,World_Index_AdjClose,0.099309,0.099309
31,DJCI_High,-0.085248,0.085248
30,DJCI_Open,-0.085133,0.085133
34,DJCI_AdjClose,-0.083129,0.083129
33,DJCI_Close,-0.083129,0.083129
